# 2. Processar Dados de Consultas

Este notebook processa o relatório de "Slip Fatura" (Consultas) da operadora e atualiza a planilha de dados.

**Entradas necessárias:**
- Arquivo `dados_atualizados.xlsx` (gerado no passo anterior de Mensalidades)
- Relatório de Slip Fatura (Consultas) da operadora (arquivo `.xls`)

**Saída:**
- Arquivo `dados_com_consultas.xlsx` (Planilha final com Mensalidades e Consultas atualizadas)

In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

### 2.1 Carregamento dos Dados
Carrega o arquivo gerado anteriormente e o novo relatório de consultas.

**Atenção:** Configure o caminho do relatório de Slip Fatura abaixo.

In [ ]:
dados = pd.read_excel('../Data/dados.xlsx', skiprows=1 ,index_col=0, sheet_name="OUT 2025")
dados['NOME'] = dados['NOME'].astype(str).str.strip()

caminho_consultas = '../Data/consultas.xls'  
consultas = pd.read_excel(caminho_consultas, index_col=False)

### 2.2 Visualização da Estrutura
Verifica as colunas para garantir que o arquivo foi lido corretamente.

In [ ]:
display(dados.head())
display(consultas.head())

### 2.3 Processamento do Relatório de Consultas
O relatório de consultas possui uma estrutura hierárquica. O código abaixo varre o arquivo identificando:
1. Linhas que iniciam um bloco de beneficiário (`Evento` começa com "Beneficiário:")
2. Linhas que contêm o total a ser cobrado (`Doc. Finan.` é "Total Crédito")

Extraímos o nome e o valor total para cada pessoa.

In [ ]:
results = []
beneficiario_name = None

# Itera sobre todas as linhas do relatório de consultas
for index, row in consultas.iterrows():
    # Verifica se a coluna 'Evento' indica o início de um beneficiário
    if row['Evento'] is not None and isinstance(row['Evento'], str) and row['Evento'].startswith('Beneficiário:'):
        # Extrai o nome formatado: "Beneficiário: NOME DO PESSOA - CÓDIGO ..."
        # Pega a parte entre "Beneficiário: " e " - "
        try:
            beneficiario_name = row['Evento'].split(' - ')[1].split(' (')[0]
            # Ajuste extra: Às vezes o nome pode vir com espaços extras
            beneficiario_name = beneficiario_name.strip()
        except IndexError:
            print(f"Aviso: Não foi possível extrair nome na linha {index}: {row['Evento']}")
            beneficiario_name = None

    # Se já temos um beneficiário identificado, procuramos a linha de total
    elif row['Doc. Finan.'] == 'Total Crédito' and beneficiario_name is not None:
        total_credito = row['Contrato Financeiro']
        
        results.append({
            'Nome Beneficiário': beneficiario_name,
            'Total Crédito': total_credito
        })
        
        # Reseta para buscar o próximo
        beneficiario_name = None

# Cria um DataFrame auxiliar com os resultados extraídos
beneficiarios = pd.DataFrame(results)

print(f"Total de beneficiários com consultas encontrados: {len(beneficiarios)}")
display(beneficiarios.head())

### 2.4 Validação de Totais (Opcional)
Exibe o valor total das consultas para conferência.

In [ ]:
total_geral = beneficiarios['Total Crédito'].sum()
print(f"Valor total das consultas processadas: R$ {total_geral:.2f}")

### 2.5 Atualização da Planilha Principal
Transfere os valores encontrados para a coluna `CONSULTA` da planilha de dados.

In [ ]:
# Inicializa a coluna CONSULTA com NaN ou 0 se preferir (opcional)
# dados['CONSULTA'] = pd.NA 

nomes_nao_encontrados = []

for index, row in beneficiarios.iterrows():
    beneficiario_name = row['Nome Beneficiário']
    total_credito = row['Total Crédito']

    # Verifica se o nome existe na planilha de dados
    if beneficiario_name in dados['NOME'].values:
        dados.loc[dados['NOME'] == beneficiario_name, 'CONSULTA'] = total_credito
    else:
        nomes_nao_encontrados.append(beneficiario_name)

display(dados.head())

### 2.6 Relatório de Nomes Não Encontrados
Lista beneficiários que tiveram consulta mas não foram achados na planilha principal.

In [ ]:
if nomes_nao_encontrados:
    print("ATENÇÃO: Os seguintes nomes do relatório de consultas NÃO foram encontrados na planilha base:")
    for nome in nomes_nao_encontrados:
        print(f"- {nome}")
else:
    print("Sucesso: Todos os beneficiários com consultas foram encontrados e atualizados.")

### 2.7 Exportação Final
Salva o arquivo completo.

In [ ]:
dados['NOME'] = dados['NOME'].astype(str).str.strip()
dados.to_excel('dados_com_consultas.xlsx', index=False)
print("Arquivo salvo como 'dados_com_consultas.xlsx'.")